# CONVERT FILES WITH DUCK_DB

In [1]:
import pandas as pd
import numpy as np
import duckdb
import os
import sys
import time 

In [ ]:
OUTPUT_PATH = r"..\\data_processed\\guo_subs_world\\subs_eu\\" 

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
import constants 

In [3]:
extra_eu = set(constants.americas + constants.asia + constants.africa + constants.oceania)

In [4]:
def time_it(func):
    """Decorator to measure the execution time of a function."""
    def wrapper(*args, **kwargs):
        start_time = time.time()  
        result = func(*args, **kwargs) 
        end_time = time.time() 
        elapsed_time = end_time - start_time  
        print(f"Execution time: {elapsed_time:.4f} seconds")  
        return result
    return wrapper

## Ownership history

In [5]:
from constants import africa, americas, oceania, asia

In [6]:
TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed"
TEMP_TABLE_KEY_FINANCIALS_DETAILED = "..\\data_processed\\key_financials_detailed_processed"

def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")

    COUNTRY_FIRMOGRAPHICS = TEMP_TABLE_FIRMOGRAPHICS + f"\\firmographics_{country}.parquet"
    COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS_DETAILED + f"\\key_financials_detailed_{country}.parquet"

    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            {year} AS year,
            firmographics.nuts2 AS subsidiary_nuts2,
            key_financials_detailed.number_of_employees
        FROM 
            '{path}' AS main

        LEFT JOIN 
            '{COUNTRY_KEY_FINANCIALS}' AS key_financials_detailed
        ON 
            main.guo_25 = key_financials_detailed.bvd_id_number
        AND 
            key_financials_detailed.year = {year}

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics
        ON 
            main.guo_25 = firmographics.bvd_id_number

        WHERE 
            main."type_of_relation" = 'GUO 25'
        AND 
            main."guo_25" LIKE '{country}%'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df


In [8]:
@time_it
def fetch_and_convert_to_excel(year, country, path, output_path):
    df = get_ownership_data(year, country, path)
    os.makedirs(output_path, exist_ok=True)

    # Split the DataFrame into 5 chunks
    chunks = 5
    indices = np.array_split(df.index, chunks)
    slices = [df.iloc[idx] for idx in indices]
    
    # Example of exporting each slice to Stata
    for i, s in enumerate(slices):
        s.to_csv(f"{output_path}\\{country}_{year}_chunk_{i+1}.csv", index=False)
    
        print(f"Data for {country} - {year} - slice_{i+1} has been converted.")
        

### Test

In [9]:
country = "AL"
year = "2017"
path = "..\\data_raw\\ownership_history\\links_2017\\*.parquet"

df = get_ownership_data(
    path=path,
    year=year,
    country=country,
)

AL - 2017...


In [ ]:
import time
import json

errors = {}
OUTPUT_PATH = r"..\\data_processed\\guo_subs_world\\subs_eu\\" 

if not os.path.exists(OUTPUT_PATH):
    print(f"Folder does not exist: {OUTPUT_PATH}")
    os.makedirs(OUTPUT_PATH, exist_ok=True)

total_start = time.time()
start_time = time.time()

for country in asia+oceania+africa+americas:
    for year in range(2007, 2023):
        year = str(year)
        try:
            path = f"..\\data_raw\\ownership_history\\links_{year}\\*.parquet"
            fetch_and_convert_to_excel(year, country, path, OUTPUT_PATH)  
            end_time = time.time()
            duration = end_time - start_time
            print(f"Data for {country} - {year} has been converted to Stata in {duration:.2f} seconds.")
        except Exception as e:
            errors[f"{country} - {year}"] = str(e)
            print(f"Error {country} - {year}")
            continue


end_time = time.time()
duration = end_time - start_time
total_end = time.time()
total_duration = total_end - total_start
print(f"Total execution time: {total_duration:.2f} seconds.")

with open("errors/errors_guos_world.json", "w") as f:
    json.dump(errors, f)

AF - 2007...
Data for AF - 2007 - slice_1 has been converted.
Data for AF - 2007 - slice_2 has been converted.
Data for AF - 2007 - slice_3 has been converted.
Data for AF - 2007 - slice_4 has been converted.
Data for AF - 2007 - slice_5 has been converted.
Execution time: 5.4008 seconds
Data for AF - 2007 has been converted to Stata in 5.40 seconds.
AF - 2008...
Data for AF - 2008 - slice_1 has been converted.
Data for AF - 2008 - slice_2 has been converted.
Data for AF - 2008 - slice_3 has been converted.
Data for AF - 2008 - slice_4 has been converted.
Data for AF - 2008 - slice_5 has been converted.
Execution time: 5.8721 seconds
Data for AF - 2008 has been converted to Stata in 11.27 seconds.
AF - 2009...
Data for AF - 2009 - slice_1 has been converted.
Data for AF - 2009 - slice_2 has been converted.
Data for AF - 2009 - slice_3 has been converted.
Data for AF - 2009 - slice_4 has been converted.
Data for AF - 2009 - slice_5 has been converted.
Execution time: 6.7599 seconds
Data

In [ ]:
# open errors
import json 

with open("errors/errors_guos_extraeu.json", "r") as f:
    errors = json.load(f)

errors

## Zip the files

In [ ]:
from utils import zip

zipper = zip.FileZipper(OUTPUT_PATH) 
zipper.zip_folder("subs_extraeu.zip")